In [1]:
# Based losely on: https://stackoverflow.com/questions/47649987/how-to-save-nltk-concordance-results-in-a-list
#https://www.kaggle.com/rtatman/color-terms-dataset#color-concrete-objects.csv
import nltk
from nltk.corpus import gutenberg
from nltk.corpus import brown
from nltk.text import ConcordanceIndex
import re
import string
from nltk.corpus import stopwords
import pandas as pd
import os
from nltk.corpus import webtext
from nltk.corpus import reuters

In [2]:
curr_dir=os.getcwd()

In [3]:
colors_concrete = pd.read_csv(curr_dir+'/color-terms-dataset/color-concrete-objects.csv')
colors_lit_nonlit = pd.read_csv(curr_dir+'/color-terms-dataset/literal-nonliteral-uses.csv')

In [4]:
fileids = gutenberg.fileids()

In [5]:
#fileids
global color_dictionary
color_dictionary={'black':[[]],
 'blue':[[]],
 'brown':[[]],
 'green':[[]],
 'grey':[[]],
 'orange':[[]],
 'pink':[[]],
 'red':[[]],
 'white':[[]],
 'yellow':[[]]}

In [6]:
global color_codes
color_codes={'black':(0, 0, 0),
 'blue':(0,0,255),
 'brown':(165,42,42),
 'green':(0,255,0),
 'grey':(128,128,128),
 'orange':(255,165,0),
 'pink':(255,192,203),
 'red':	(255,0,0),
 'white':(255,255,255),
 'yellow':(255,255,0)}

In [7]:
# Credit to:https://stackoverflow.com/questions/47649987/how-to-save-nltk-concordance-results-in-a-list
#By inspecting the source of ConcordanceIndex, we can see that results are printed to stdout. If redirecting stdout to a file is not an option, you have to reimplement the ConcordanceIndex.print_concordance such that it returns the results rather than printing it to stdout.

def concordance(ci, word, width=200, lines=100):
    """
    Rewrite of nltk.text.ConcordanceIndex.print_concordance that returns results
    instead of printing them. 

    See:
    http://www.nltk.org/api/nltk.html#nltk.text.ConcordanceIndex.print_concordance
    """
    half_width = (width - len(word) - 2) // 2
    context = width // 4 # approx number of words of context

    results = []
    offsets = ci.offsets(word)
    if offsets:
        lines = min(lines, len(offsets))
        for i in offsets:
            if lines <= 0:
                break
            left = (' ' * half_width +
                    ' '.join(ci._tokens[i-context:i]))
            right = ' '.join(ci._tokens[i+1:i+context])
            left = left[-half_width:]
            right = right[:half_width]
            results.append('%s %s %s' % (left, ci._tokens[i], right))
            lines -= 1

    return results

In [8]:
def remove_stop_punct(text):
    tokenized = [w.lower() for w in text if not re.fullmatch('[' + string.punctuation + ']+', w)]
    #print(tokenized)
    
    processed_data=[w for w in tokenized if w not in stopwords.words("english")]
    
    output = [re.sub(r'\d+', '', w) for w in processed_data] #remove numbers
    #'hello467'.translate(None, '0123456789')
    output = [word.lower() for word in output if word.isalpha()]
    output = [word.encode('ascii',errors='ignore').decode() for word in output]
    output = [word for word in output if len(word)!=0]
    return output
    #return ''.join(tokenized)

In [9]:
#emma = nltk.Text(nltk.corpus.gutenberg.words('austen-emma.txt'))
#ci = ConcordanceIndex(emma)
#results = concordance(ci, 'red')
#s = [x for x in emma.concordance("red", width=2)]
#brown.words(categories='news')
#type(x)
remove_stop_punct(nltk.word_tokenize('stop and words123,! äää äé é é'))

['stop', 'words']

In [10]:
#results[0]
#col='red'
#re.sub(col, '', results[0])

In [11]:
for ids in fileids:
    #print(ids)
    book_words = nltk.Text(nltk.corpus.gutenberg.words(ids))
    ci = ConcordanceIndex(book_words)
    for (col, val) in color_dictionary.items():
        results = concordance(ci, col)
        for conc in results:
            #print(type(r))
            x = remove_stop_punct(nltk.word_tokenize(re.sub(col, '', conc)))
            if x!='' or x is not None:
                color_dictionary[col][0].append(' '.join(x))
print('Done.\n')

Done.



In [12]:
#for fileid in webtext.fileids():
     #print(fileid, nltk.Text(webtext.raw(fileid)[:65]), '...')

In [13]:
def get_context(corp):
    for fileid in corp.fileids():
        book_words = nltk.Text(nltk.word_tokenize(corp.raw(fileid)))
        #book_words = nltk.Text(nltk.word_tokenize(te))
    
        #print(book_words)
        ci = ConcordanceIndex(book_words)
        #print(ci.print_concordance('black'))
        for (col, _) in color_dictionary.items():
            results = concordance(ci, col)
            for conc in results:
                #print(type(r))
                x = remove_stop_punct(nltk.word_tokenize(re.sub(col, '', conc)))
                if x!='' or x is not None:
                    color_dictionary[col][0].append(' '.join(x))
    print('Done.\n')

In [14]:
get_context(webtext)

Done.



In [15]:
get_context(reuters)

Done.



In [16]:
for (color, value) in color_dictionary.items():
    nf = True 
    while nf:
        nf=False
        for v in value:
            if len(v)==0:
                value.remove(v)
                nf=True
for (color, value) in color_dictionary.items():
    print(color, '-', len(value[0]))
    

black - 610
blue - 299
brown - 154
green - 351
grey - 169
orange - 76
pink - 51
red - 429
white - 771
yellow - 159


In [17]:
print(color_dictionary.get('orange'))

[['veral years take granted observed sir walter face cuffs capes livery mr shepherd hastened assure admiral croft w', 'ne fat currants mouth made rose coloured sugar gay little cap sugar candy little old woman rolled dressed pinched', 'even reference daffodil innocent lonely yellow never heard though might turn quite unconsciously green tops one might say', 'consent even continue round astronomers always telling us shaped like like egg like german sausage beat old world like bladder', 'overlooked mediterranean covered awning fenced little lemon trees waiters white aprons already laying white tables insignia early', 'colour curiously attractive partly tropical reminded one vaguely blood ruddy sanguine yellow way', 'er pause susan poured broth basin strewed bright marigolds looked tempting tasted added little salt li', 'rich satin bag covered gold netting mr mansfield oppidan white purple dress trimmed silver rich satin bag purple silver carrying elegant poles', 'hite gold dress humble o

In [18]:
color_dictionary.keys()


dict_keys(['black', 'blue', 'brown', 'green', 'grey', 'orange', 'pink', 'red', 'white', 'yellow'])

In [19]:
#df_colors = pd.DataFrame.from_dict(color_dictionary, orient='index', columns=['context'])
df_colors = pd.DataFrame.from_dict(color_dictionary, orient='index', columns=['context'])

In [20]:


#print (color_dictionary)
#for (name, tuples) in color_dictionary.items():
#    print(name,'-',tuples)
    #for ()
df_colors['code']=''
df_colors.reset_index(level=0, inplace=True)
df_colors.rename(index=str, columns={"index": "name", "context":"context", "code":"code"}, inplace=True)
df_colors


,name,context,code
0,black,[self denying generous friendship always wishe...,
1,blue,[f sort emma particularly admired short plump ...,
2,brown,[ection oh could originals portraits wainscot ...,
3,green,[woody varieties cheerful village lyme pinny c...,
4,grey,[han remembered regular pleasing beauty eyes d...,
5,orange,[veral years take granted observed sir walter ...,
6,pink,[onnet bring one little baskets hanging arm pr...,
7,red,[obliged repeated could believed mr knightley ...,
8,white,[sky appearance overcharged want milder air pr...,
9,yellow,[days harriet oh sweet house beautiful curtain...,


In [21]:
df_colors['red']=''
df_colors['green']=''
df_colors['blue']=''

In [22]:
for i in range(df_colors.shape[0]):
    df_colors.iloc[i,2]=color_codes[df_colors.iloc[i,0]]
    df_colors.iloc[i,3]=color_codes[df_colors.iloc[i,0]][0]
    df_colors.iloc[i,4]=color_codes[df_colors.iloc[i,0]][1]
    df_colors.iloc[i,5]=color_codes[df_colors.iloc[i,0]][2]

In [23]:
color_codes['red']

(255, 0, 0)

In [24]:
a=(1,2,3)
a[0]


1

In [25]:
type(df_colors.context)

pandas.core.series.Series

In [26]:
df_colors.context[0]

['self denying generous friendship always wished promoted match morning work want miss taylor would felt every hour every day r',
 'er lips arrived carriage turned step let mr elton spruce smiling instantly emma thought pleasure change subject',
 'town next week see latest dare say impatient gentleman thing done likely morrow saturday',
 'laughed fears could cured wishing would part mare expressed coming common certainty aunt',
 'gotten think glad see mrs weston would ease mare blameless right named mrs churchill cause',
 'conducted though present time summer wearing ribbons wife could admit worth thinking disgrace h',
 'ming apologize spend evening first idea mary quite ready affronted louisa made right saying sh',
 'gone could quit mansion house look adieu cottage dripping comfortless veranda even notice misty glasses last humble te',
 'punction totally beyond reach sentiment justice compassion oh heart hollow anne astonished air exclamation wonder made pa',
 'ond reach sentiment jus

In [27]:
df_colors.to_csv(curr_dir+'/color_data.csv')

In [28]:
#df_colors_final=pd.DataFrame({'names':'', 'context':'', 'code':''})

In [29]:
df_colors.shape

(10, 6)

In [30]:
temp_name=[]
temp_context=[]
temp_code=[]
temp_red=[]
temp_blue=[]
temp_green=[]
index=0
for i in range(df_colors.shape[0]):
    index+=1
    for j in range(len(df_colors.iloc[i,1])):
            temp_context.append(df_colors.iloc[i,1][j])
            temp_name.append(df_colors.iloc[i,0])
            temp_code.append(df_colors.iloc[i,2])
            temp_red.append(df_colors.iloc[i,3])
            temp_green.append(df_colors.iloc[i,4])
            temp_blue.append(df_colors.iloc[i,5])
            index+=1
temp_context
temp_name
temp_code
temp_red
temp_blue
temp_green
df_colors_final=pd.DataFrame.from_dict({"name":temp_name, "context":temp_context, "code":temp_code, "red": temp_red,
                                       "green":temp_green, "blue":temp_blue})

df_colors_final

,name,context,code,red,green,blue
0,black,self denying generous friendship always wished...,"(0, 0, 0)",0,0,0
1,black,er lips arrived carriage turned step let mr el...,"(0, 0, 0)",0,0,0
2,black,town next week see latest dare say impatient g...,"(0, 0, 0)",0,0,0
3,black,laughed fears could cured wishing would part m...,"(0, 0, 0)",0,0,0
4,black,gotten think glad see mrs weston would ease ma...,"(0, 0, 0)",0,0,0
5,black,conducted though present time summer wearing r...,"(0, 0, 0)",0,0,0
6,black,ming apologize spend evening first idea mary q...,"(0, 0, 0)",0,0,0
7,black,gone could quit mansion house look adieu cotta...,"(0, 0, 0)",0,0,0
8,black,punction totally beyond reach sentiment justic...,"(0, 0, 0)",0,0,0
9,black,ond reach sentiment justice compassion oh hear...,"(0, 0, 0)",0,0,0


In [31]:

for fileid in reuters.fileids():
     print(fileid, nltk.Text(reuters.raw(fileid)[:65]), '...')

test/14826 <Text: A S I A N   E X...> ...
test/14828 <Text: C H I N A   D A...> ...
test/14829 <Text: J A P A N   T O...> ...
test/14832 <Text: T H A I   T R A...> ...
test/14833 <Text: I N D O N E S I...> ...
test/14839 <Text: A U S T R A L I...> ...
test/14840 <Text: I N D O N E S I...> ...
test/14841 <Text: S R I   L A N K...> ...
test/14842 <Text: W E S T E R N  ...> ...
test/14843 <Text: S U M I T O M O...> ...
test/14844 <Text: S U B R O T O  ...> ...
test/14849 <Text: B U N D E S B A...> ...
test/14852 <Text: B O N D   C O R...> ...
test/14854 <Text: C H I N A   I N...> ...
test/14858 <Text: J A P A N   M I...> ...
test/14859 <Text: A M A T I L   P...> ...
test/14860 <Text: B O W A T E R  ...> ...
test/14861 <Text: U . K .   M O N...> ...
test/14862 <Text: S O U T H   K O...> ...
test/14863 <Text: F I N N S   A N...> ...
test/14865 <Text: C R A   S O L D...> ...
test/14867 <Text: G E R M A N   I...> ...
test/14872 <Text: B O W A T E R  ...> ...
test/14873 <Text: C I T I B A N K.

test/15579 <Text: F R E D D I E  ...> ...
test/15580 <Text: A R G E N T I N...> ...
test/15581 <Text: V W   S A Y S  ...> ...
test/15582 <Text: Y U G O S L A V...> ...
test/15583 <Text: F E D E R A T E...> ...
test/15584 <Text: P R I C E S   G...> ...
test/15585 <Text: A L E X .   B R...> ...
test/15590 <Text: H I G H L A N D...> ...
test/15591 <Text: L V I   G R O U...> ...
test/15593 <Text: T E X A S   A M...> ...
test/15594 <Text: V O R N A D O  ...> ...
test/15595 <Text: B R E N D A   M...> ...
test/15596 <Text: P H O E N I X  ...> ...
test/15597 <Text: H E A L T H   A...> ...
test/15598 <Text: T .   R O W E  ...> ...
test/15600 <Text: G E N E R A L  ...> ...
test/15601 <Text: H E R I T A G E...> ...
test/15602 <Text: N A T I O N A L...> ...
test/15603 <Text: U . K .   C H A...> ...
test/15605 <Text: R E X H A M   &...> ...
test/15607 <Text: E X X O N   & l...> ...
test/15610 <Text: F H L B B   C H...> ...
test/15613 <Text: & l t ; S T R A...> ...
test/15615 <Text: F E D   M A Y  .

test/16394 <Text: F R A N K L I N...> ...
test/16395 <Text: G E N E R A L  ...> ...
test/16396 <Text: F R A N K L I N...> ...
test/16398 <Text: G A F   & l t ;...> ...
test/16399 <Text: N A T I O N A L...> ...
test/16400 <Text: L A S E R   P H...> ...
test/16401 <Text: E A S T E R N  ...> ...
test/16402 <Text: I T A L Y ' S  ...> ...
test/16403 <Text: M C G R A W - H...> ...
test/16404 <Text: O A K   O N D U...> ...
test/16405 <Text: P E O P L E S  ...> ...
test/16406 <Text: R E V L O N   G...> ...
test/16407 <Text: U . S .   T R U...> ...
test/16409 <Text: ( C O R R E C T...> ...
test/16410 <Text: B I O T E C H  ...> ...
test/16415 <Text: P O R T U G U E...> ...
test/16417 <Text: M Y E R S   I N...> ...
test/16418 <Text: G T E   C O R P...> ...
test/16419 <Text: R E S D E L   &...> ...
test/16420 <Text: M I D L A N T I...> ...
test/16421 <Text: P E O P L E ' S...> ...
test/16422 <Text: M I C R O   G E...> ...
test/16424 <Text: G T E   C O R P...> ...
test/16426 <Text: G T E   P O S T.

test/16743 <Text: B a n k   o f  ...> ...
test/16744 <Text: B A N K   O F  ...> ...
test/16745 <Text: M I Y A Z A W A...> ...
test/16746 <Text: J a p a n   M a...> ...
test/16747 <Text: N O   T A L K S...> ...
test/16749 <Text: H A I T I A N  ...> ...
test/16750 <Text: C H I N A   T O...> ...
test/16751 <Text: T A I W A N   I...> ...
test/16752 <Text: T A I W A N   L...> ...
test/16753 <Text: C H I N A   F A...> ...
test/16754 <Text: B a n k   o f  ...> ...
test/16755 <Text: J A P A N   M A...> ...
test/16759 <Text: S O U T H   K O...> ...
test/16761 <Text: B A N K   O F  ...> ...
test/16762 <Text: S A U D I   A R...> ...
test/16763 <Text: U . S .   U R G...> ...
test/16765 <Text: 6 0 , 0 0 0   T...> ...
test/16766 <Text: P A P E R   S A...> ...
test/16767 <Text: S O U T H   K O...> ...
test/16770 <Text: J A P A N ' S  ...> ...
test/16772 <Text: I N D I A ' S  ...> ...
test/16774 <Text: J A P A N   D E...> ...
test/16775 <Text: C H I N A   T R...> ...
test/16776 <Text: T H A I   V E N.

test/17853 <Text: L A D D   & l t...> ...
test/17857 <Text: C I S   & l t ;...> ...
test/17860 <Text: W A S T E   M A...> ...
test/17863 <Text: B U R L I N G T...> ...
test/17864 <Text: E D E L M A N  ...> ...
test/17866 <Text: C O N T ' L   M...> ...
test/17867 <Text: A M E R I B A N...> ...
test/17870 <Text: V E N E Z U E L...> ...
test/17871 <Text: U . S .   T R A...> ...
test/17872 <Text: C O U R T   U P...> ...
test/17873 <Text: H A R C O U R T...> ...
test/17874 <Text: V E N E Z U E L...> ...
test/17875 <Text: S O U T H   K O...> ...
test/17876 <Text: E Q U I T I C O...> ...
test/17877 <Text: V E N E Z U E L...> ...
test/17878 <Text: E S S O   R A I...> ...
test/17880 <Text: S O U T H   K O...> ...
test/17881 <Text: T o k y o   -  ...> ...
test/17883 <Text: N O   R E A S O...> ...
test/17886 <Text: J A P A N   O I...> ...
test/17887 <Text: F U J I T S U  ...> ...
test/17888 <Text: J A P A N   R E...> ...
test/17892 <Text: M E E S   N E W...> ...
test/17894 <Text: M A L A Y S I A.

...

training/8120 <Text: W O O L W O R T...> ...
training/8123 <Text: B e l l   R e s...> ...
training/8125 <Text: W O O L W O R T...> ...
training/8126 <Text: B E L L   R E S...> ...
training/8130 <Text: A R I A D N E  ...> ...
training/8131 <Text: E X X O N   O F...> ...
training/8132 <Text: T A I W A N   T...> ...
training/8134 <Text: A U S T R A L I...> ...
training/8135 <Text: G A T T   M E E...> ...
training/8137 <Text: G E R M A N   B...> ...
training/8138 <Text: P H I L I P P I...> ...
training/8140 <Text: J A P A N E S E...> ...
training/8141 <Text: J A P A N   S A...> ...
training/8144 <Text:   B u n d e s b...> ...
training/8145 <Text: B U N D E S B A...> ...
training/8147 <Text: U . K .   M O N...> ...
training/8149 <Text: I N D O N E S I...> ...
training/8151 <Text: A U S T R A L I...> ...
training/8153 <Text: J A P A N   E C...> ...
training/8155 <Text: B O O K E R   P...> ...
training/8156 <Text: P H I L I P P I...> ...
training/8158 <Text: B U R M A   S A...> ...
training/8

training/8612 <Text: W o o l w o r t...> ...
training/8613 <Text: C H I N A   S A...> ...
training/8614 <Text: W O O L W O R T...> ...
training/8615 <Text: T M O C   T E L...> ...
training/8616 <Text: S A N T O S   L...> ...
training/8617 <Text: W I L L I A M S...> ...
training/8618 <Text: W I L L I A M S...> ...
training/8619 <Text: S A N T O S   L...> ...
training/8621 <Text: D O U B T S   A...> ...
training/8623 <Text: A U S T R A L I...> ...
training/8624 <Text: G E R M A N   F...> ...
training/8628 <Text: N I P P O N   L...> ...
training/8629 <Text: U . K .   M O N...> ...
training/8630 <Text: S A N T O S   S...> ...
training/8632 <Text: P   A N D   O  ...> ...
training/8635 <Text: C A N A D A   O...> ...
training/8636 <Text: T A I W A N   I...> ...
training/8637 <Text: P H I L I P P I...> ...
training/8638 <Text: D R O U G H T  ...> ...
training/8641 <Text: M I Y A Z A W A...> ...
training/8643 <Text: B r i t i s h  ...> ...
training/8644 <Text: B R I T I S H  ...> ...
training/8

training/8906 <Text: O L I V E T T I...> ...
training/8908 <Text: E N S E R C H  ...> ...
training/8909 <Text: N E S T L E   T...> ...
training/8911 <Text: F E D ' S   H E...> ...
training/8912 <Text: M E T R O P O L...> ...
training/8914 <Text: G O O D Y E A R...> ...
training/8915 <Text: M I C R O S I Z...> ...
training/8916 <Text: & l t ; P A U L...> ...
training/8918 <Text: C Y A C Q   A M...> ...
training/8919 <Text: G O R D O N   J...> ...
training/8922 <Text: F E D ' S   H E...> ...
training/8925 <Text: T A J O N   R A...> ...
training/8928 <Text: C E N T E R I O...> ...
training/8930 <Text: S U F F I E L D...> ...
training/8932 <Text: C Y A C Q   C U...> ...
training/8933 <Text: T I N   P A C T...> ...
training/8935 <Text: S I G M A   R E...> ...
training/894 <Text: J A P A N   M O...> ...
training/8940 <Text: K O D A K   & l...> ...
training/8941 <Text: H E A V Y   S I...> ...
training/8943 <Text: I L L I N O I S...> ...
training/8944 <Text: C O N R A C   &...> ...
training/89

training/9372 <Text: P E P   B O Y S...> ...
training/9373 <Text: H O V N A N I A...> ...
training/9374 <Text: N Y   T R A D E...> ...
training/9377 <Text: J O H N S O N  ...> ...
training/9378 <Text: S U N S T A R  ...> ...
training/938 <Text: C O M A L C O  ...> ...
training/9381 <Text: H A D S O N   &...> ...
training/9382 <Text: R E U T E R S  ...> ...
training/9383 <Text: T C B Y   E N T...> ...
training/9385 <Text: R A Y T H E O N...> ...
training/9388 <Text: P . H .   G L A...> ...
training/9389 <Text: L I F E   O F  ...> ...
training/939 <Text: U L T R A M A R...> ...
training/9392 <Text: C O N S O L I D...> ...
training/9393 <Text: T H A I   S U G...> ...
training/9397 <Text: R E I C H H O L...> ...
training/9398 <Text: N Y   T R A D E...> ...
training/9399 <Text: R O S S   S T O...> ...
training/94 <Text: U . S .   B A N...> ...
training/940 <Text: H O N G   K O N...> ...
training/9403 <Text: N A V I S T A R...> ...
training/9405 <Text: C O R R E C T E...> ...
training/9406 <

training/9681 <Text: E X C H A N G E...> ...
training/9682 <Text: C O N S O L I D...> ...
training/9686 <Text: T R E A S U R Y...> ...
training/9689 <Text: B A K E R   S A...> ...
training/969 <Text: N E I T H E R  ...> ...
training/9697 <Text: B A K E R   S E...> ...
training/9698 <Text: S u m i t a   s...> ...
training/9699 <Text: S u m i t a   s...> ...
training/97 <Text: A S C S   T E R...> ...
training/9700 <Text: N . Z .   M O N...> ...
training/9701 <Text: F U R T H E R  ...> ...
training/9704 <Text: E C   L I N K S...> ...
training/9705 <Text: V I E T N A M '...> ...
training/9706 <Text: L I N D E   T U...> ...
training/9707 <Text: L I N D E   A G...> ...
training/971 <Text: U N I L E V E R...> ...
training/9712 <Text: T R A D E   S U...> ...
training/9718 <Text: G U L F   E S C...> ...
training/972 <Text: U . S .   L E A...> ...
training/9720 <Text: M I Y A Z A W A...> ...
training/9723 <Text: H o e c h s t  ...> ...
training/9727 <Text: B A N K   O F  ...> ...
training/9728 <

In [32]:
"""te = "the night is black home the night is black home the night is black home the night is black home the night is black home the night is black home the night is black home the night is black home"
nltk.Text(te)
ci = ConcordanceIndex(te)
concordance(ci, "black")"""

'te = "the night is black home the night is black home the night is black home the night is black home the night is black home the night is black home the night is black home the night is black home"\nnltk.Text(te)\nci = ConcordanceIndex(te)\nconcordance(ci, "black")'

In [33]:
"""for ids in fileids:
    #print(ids)
    #book_words = nltk.Text(nltk.corpus.gutenberg.words(ids))
    for fileid in reuters.fileids():
        book_words = nltk.Text(nltk.word_tokenize(reuters.raw(fileid)))
        #book_words = nltk.Text(nltk.word_tokenize(te))
        
        #print(book_words)
        ci = ConcordanceIndex(book_words)
        #print(ci.print_concordance('black'))
        for (col, _) in color_dictionary.items():
            results = concordance(ci, col)
            if len(results)!=0:
                print(results)
                    #color_dictionary[col][0].append(' '.join(x))"""

"for ids in fileids:\n    #print(ids)\n    #book_words = nltk.Text(nltk.corpus.gutenberg.words(ids))\n    for fileid in reuters.fileids():\n        book_words = nltk.Text(nltk.word_tokenize(reuters.raw(fileid)))\n        #book_words = nltk.Text(nltk.word_tokenize(te))\n        \n        #print(book_words)\n        ci = ConcordanceIndex(book_words)\n        #print(ci.print_concordance('black'))\n        for (col, _) in color_dictionary.items():\n            results = concordance(ci, col)\n            if len(results)!=0:\n                print(results)\n                    #color_dictionary[col][0].append(' '.join(x))"

#### Adding the concrete colors dataset

----

In [34]:
colors_concrete.columns

Index(['object', 'color'], dtype='object')

In [35]:
colors_lit_nonlit.columns

Index(['color', 'noun', 'literal_or_nonliteral'], dtype='object')

In [36]:
#flip columns
col = ['color', 'object']
colors_concrete = colors_concrete[col]

In [37]:
colors_concrete = colors_concrete.rename(index=str, columns={"color": "name", "object": "context"})

In [38]:
colors_concrete[colors_concrete.name=='red']

,name,context
3,red,blood
10,red,cherry
45,red,strawberry


In [39]:
#dataframe drop last lit non lit column
colors_lit_nonlit = colors_lit_nonlit.drop(['literal_or_nonliteral'], axis=1)
#rename 'noun' column 'context'
colors_lit_nonlit = colors_lit_nonlit.rename(index=str, columns={"color": "name", "noun": "context"})

In [40]:
colors_lit_nonlit[colors_lit_nonlit.name=='red'].head(3)

,name,context
290,red,meat
291,red,disc
292,red,tower


In [41]:
frames = [colors_concrete, colors_lit_nonlit]
colors_concrete = pd.concat(frames)
colors_concrete.shape

(394, 2)

In [42]:
#augment dataset
for name in set(colors_concrete.name):
    d = colors_concrete[colors_concrete.name==name]
    for i in range(0,len(d.context)-2, 2):
        #print(name,'--',d.context.iloc[i]+' '+d.context.iloc[i+1])
        colors_concrete = colors_concrete.append({'name': name, 'context': d.context.iloc[i]+' '+d.context.iloc[i+1]}, ignore_index=True)
colors_concrete.shape

(583, 2)

In [43]:
print(set(colors_concrete.name))
print(set(color_codes.keys()))
print("All colors match: ", len(set(colors_concrete.name)-set(color_codes.keys()))==0)

{'white', 'red', 'pink', 'green', 'black', 'orange', 'grey', 'brown', 'blue', 'yellow'}
{'white', 'red', 'pink', 'green', 'black', 'orange', 'grey', 'blue', 'brown', 'yellow'}
All colors match:  True


In [44]:
colors_concrete['code']=''

In [45]:
colors_concrete['red']=''
colors_concrete['green']=''
colors_concrete['blue']=''

In [46]:
print(colors_concrete.iloc[2,0])
print(color_codes[colors_concrete.iloc[2,0]])

brown
(165, 42, 42)


In [47]:
for i in range(colors_concrete.shape[0]):
    colors_concrete.iloc[i,2]=color_codes[colors_concrete.iloc[i,0]]
    colors_concrete.iloc[i,3]=color_codes[colors_concrete.iloc[i,0]][0]
    colors_concrete.iloc[i,4]=color_codes[colors_concrete.iloc[i,0]][1]
    colors_concrete.iloc[i,5]=color_codes[colors_concrete.iloc[i,0]][2]

In [48]:
colors_concrete.head(10)

,name,context,code,red,green,blue
0,grey,ash,"(128, 128, 128)",128,128,128
1,yellow,banana,"(255, 255, 0)",255,255,0
2,brown,beaver,"(165, 42, 42)",165,42,42
3,red,blood,"(255, 0, 0)",255,0,0
4,green,broccoli,"(0, 255, 0)",0,255,0
5,brown,cardboard,"(165, 42, 42)",165,42,42
6,orange,carrot,"(255, 165, 0)",255,165,0
7,white,cauliflower,"(255, 255, 255)",255,255,255
8,brown,cello,"(165, 42, 42)",165,42,42
9,white,chalk,"(255, 255, 255)",255,255,255


In [49]:
colors_concrete.shape

(583, 6)

In [50]:
#concatenate frames
frames=[colors_concrete, df_colors_final, colors_concrete]
df_colors_final = pd.concat(frames)

In [54]:
df_colors_final= df_colors_final.dropna()

In [58]:
df_colors_final = df_colors_final[df_colors_final.context.apply(lambda x: x !="")] #drop samples with empty context

In [59]:
df_colors_final.to_csv(curr_dir+'/color_data_final.csv')

In [60]:
df_colors_final.shape


(4235, 6)

In [61]:
df_colors_final.head(50)

,name,context,code,red,green,blue
0,grey,ash,"(128, 128, 128)",128,128,128
1,yellow,banana,"(255, 255, 0)",255,255,0
2,brown,beaver,"(165, 42, 42)",165,42,42
3,red,blood,"(255, 0, 0)",255,0,0
4,green,broccoli,"(0, 255, 0)",0,255,0
5,brown,cardboard,"(165, 42, 42)",165,42,42
6,orange,carrot,"(255, 165, 0)",255,165,0
7,white,cauliflower,"(255, 255, 255)",255,255,255
8,brown,cello,"(165, 42, 42)",165,42,42
9,white,chalk,"(255, 255, 255)",255,255,255
